[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/generation/langchain/langgraph/02-ollama-langgraph-agent/02-ollama-langgraph-agent.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/generation/langchain/langgraph/02-ollama-langgraph-agent/02-ollama-langgraph-agent.ipynb)

# Ollama LangGraph Agent

LangGraph is one of the most powerful frameworks for build AI agents, and Ollama one of the most popular frameworks for running local LLMs. Bringing both together allows us to run agentic workflows at little-to-no cost. In this example we will see how.

We recommend running this locally (ideally on Apple silicon). For environment setup instructions you can refer to the README found in this directory.

If using Colab, you should run the following installs (no need to run if installing locally with `poetry`):

```
!pip install -qU \
    langchain==0.2.12 \
    langchain-core==0.2.29 \
    langgraph==0.2.3 \
    langchain-ollama==0.1.1 \
    semantic-router==0.0.61 \
    pyppeteer==2.0.0 \
    nest-asyncio==1.6.0 \
    praw==7.7.1
```

In [ ]:
!conda install -n .conda ipykernel --update-deps --force-reinstall

: 

In [1]:
!pip install -qU \
    langchain==0.2.12 \
    langchain-core==0.2.29 \
    langgraph==0.2.3 \
    langchain-ollama==0.1.1 \
    semantic-router==0.0.61 \
    pyppeteer==2.0.0 \
    nest-asyncio==1.6.0 \
    praw==7.7.1

## Reddit Search Tool

We first need to sign up for the Reddit API, you can refer to the first few minutes of [this tutorial](https://www.youtube.com/watch?v=FdjVoOf9HN4) if you want a full walkthrough, but tldr;

1. Go to [App Preferences](https://www.reddit.com/prefs/apps) and click **_create another app..._** at the bottom.
2. Fill out required details, make sure to select *script* for the application type then click **_create app_**.
3. Fill out the next cell with `client_id` (your *personal use script*) and `client_secret` (your *secret* key).

In [1]:
import praw

reddit = praw.Reddit(
    client_id="---",  # personal use script
    client_secret="---",  # secret
    user_agent="search-tool"  # name of application
)

We'll be pulling in submission threads from Reddit that include user's restaurant recommendations (or just other info we search for). From the submission threads we need:

* Submission title
* Submission first text / description
* A few of the top voted comments

To organize this information we can create a pydantic class to structure the needed data:

In [21]:
from pydantic import BaseModel

class Rec(BaseModel):
    title: str
    description: str
    comments: list[str]

    def __str__(self):
        """LLM-friendly string representation of the recommendation(s)."""
        comments_str = '\n'.join(self.comments)
        return f"Title: {self.title}\nDescription: {self.description}\nComments:\n{comments_str}"


Now we setup the retrieval logic for an example query `"best pizza in rome"`:

In [3]:
from praw.models import Comment

# search across all subreddits for pizza recommendations
results = reddit.subreddit("all").search("best pizza in rome")
recs = []
for submission in results:
    title = submission.title
    description = submission.selftext
    # we only get comments with 20 or more upvotes
    comments = []
    for comment in submission.comments.list():
        if isinstance(comment, Comment) and comment.ups >= 20:
            author = comment.author.name if comment.author else "unknown"
            comments.append(f"{author} (upvotes: {comment.ups}): {comment.body}")
    # and of these, we only need 3
    comments = comments[:3]
    # if there are enough comments (ie 3), we add the recommendation to our list
    if len(comments) == 3:
        print(title)
        recs.append(Rec(title=title, description=description, comments=comments))
    if len(recs) == 3:
        # stop after getting 3 recommendations
        break

Best pizza in Rome?
Visited Rome and had one of the best pizzas of my life
Since pizza is an American food, I'm willing to bet the best pizza is in America.


Let's see what we have:

In [4]:
print("\n===\n".join([str(rec) for rec in recs]))

Title: Best pizza in Rome?
Description: I was a little disappointed after my first experience tasting pizza after pasta and gelato were ridiculously amazing. What do you recommend? 
Comments:
miclee15 (upvotes: 23): American here.  I think if OP is from the USA, Rome pizza needs to be approached differently.  I’m from NY where we think that is the best pizza in the US, people from Chicago will disagree.  Set aside the preconceived notion of what great pizza should be and enjoy the variety and flavors.   I’m in Rome now.  Went to Antico Forno Roscioli and had the most amazing porcetta pizza with potatoes on top.  I still love a NYC slice but Rome pizza is incredible at some places.  Edited for spelling
Sisyphus_Rock530 (upvotes: 29): 

- **Pizzeria da Remo** a Testaccio, nota per la sua base sottile e croccante, è molto popolare tra i romani. https://www.romeing.it/best-pizza-in-rome/).


- **Emma** vicino Campo de' Fiori, famosa per la sua pizza a crosta sottile e ingredienti di alta q

Let's put all of this together into a single `tool` that our LLM will be connected to for function calling.

In [22]:
import random

def linkedin_candidate_scrapper(query: str) -> list[Rec]:
    """
    Provides access to a LinkedIn candidate scrapping tool.
    This function simulates a candidate search by returning a list
    of random candidate recommendations. In a real implementation, it
    would connect to LinkedIn's API or use web scraping techniques
    to retrieve candidate profiles.
    """
    sample_names = [
        "Alice Johnson", 
        "Bob Smith", 
        "Carol Williams", 
        "David Brown", 
        "Ella Davis"
    ]
    sample_descriptions = [
        "Experienced software engineer with expertise in Python and Machine Learning.",
        "Results-driven project manager with a strong background in tech and operations.",
        "Creative UX/UI designer with a knack for innovative digital solutions.",
        "Data analyst skilled in SQL, Python, and dashboard reporting.",
        "Sales professional with a proven track record in exceeding targets."
    ]
    print(f"Searching for candidates with query: {query}")
    candidates = []
    # create 3 random candidate recommendations
    for _ in range(3):
        idx = random.randint(0, len(sample_names) - 1)
        candidate = Rec(
            title=sample_names[idx],
            description=sample_descriptions[idx],
            comments=[f"Candidate {sample_names[idx]} demonstrates strong potential in their field."]
        )
        candidates.append(candidate)
    return candidates

# example usage:
out = linkedin_candidate_scrapper(query="software")
print(out)


Searching for candidates with query: software
[Rec(title='Ella Davis', description='Sales professional with a proven track record in exceeding targets.', comments=['Candidate Ella Davis demonstrates strong potential in their field.']), Rec(title='Carol Williams', description='Creative UX/UI designer with a knack for innovative digital solutions.', comments=['Candidate Carol Williams demonstrates strong potential in their field.']), Rec(title='Carol Williams', description='Creative UX/UI designer with a knack for innovative digital solutions.', comments=['Candidate Carol Williams demonstrates strong potential in their field.'])]


### Final Answer "Tool"

Alongside our web search tool we will have a final tool called `final_answer`. The final answer tool will be called whenever the LLM has finished pulling info from the other two tools and is ready to provide a *final answer* to the user.

In [2]:
def final_answer(answer: str, phone_number: str = "", address: str = ""):
    """Returns a natural language response to the user. There are four sections 
    to be returned to the user, those are:
    - `answer`: the final natural language answer to the user's question, should provide as much context as possible.
    - `phone_number`: the phone number of top recommended restaurant (if found).
    - `address`: the address of the top recommended restaurant (if found).
    """
    return {
        "answer": answer,
        "phone_number": phone_number,
        "address": address,
    }

## Graph Construction

We are using LangGraph to create an agentic graph-based flow. To construct the graph we will use:

* **Agent State**: An object persisted through every step in the graph, used to provide input to nodes, and to store output from nodes to be used in later nodes or in our final output.
* **Local LLM**: We are using a local LLM (`llama-3.1:8b`) via Ollama. For tool use we turn on _JSON mode_ to reliably output parsible JSON.
* **Tools**: The tools our LLM can use, these allow use of the functions `search` and `final_answer`.
* **Graph Nodes**: We wrap our logic into components that allow it to be used by LangGraph, these consume and output the *Agent State*.


### Agent State

In [3]:
from typing import TypedDict, Annotated, List, Union
from langchain_core.agents import AgentAction
from langchain_core.messages import BaseMessage
import operator


class AgentState(TypedDict):
    input: str
    chat_history: list[BaseMessage]
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]
    output: dict[str, Union[str, List[str]]]

### LLM and Tools

The LLM acts as our decision maker and generator of our final output, we will later call this component the `oracle` as our *decision-maker*. For this we are using Ollama and `Llama 3.1`, once initialized we integrate it into a runnable pipeline of our Oracle. The system prompt for our `oracle` will be:

In [ ]:
system_prompt = """You are the oracle: a specialized recruiter agent and decision-maker.
Given the user's hiring request, decide how to proceed using the available tools.

Objective:
- Identify and recommend the best candidate(s) for the role.
- For each recommended candidate include: name/title, key skills, years of experience, location (if known), a concise rationale for fit, contact details if available, and suggested next steps for outreach/interview.

Tool-calling rules:
- When using a tool, output ONLY one JSON object and NOTHING else, exactly matching this pattern:
{
    "name": "<tool_name>",
    "parameters": {"<param_key>": <param_value>}
}
- Use at most one tool per turn.
- You may call the search tool (linkedin_candidate_scrapper) up to 3 times total.
- After any use of the search tool, you MUST call the final_answer tool to produce the human-facing summary and recommendations.
- If the user asks something unrelated to recruiting/hiring or requests a direct answer, call final_answer directly.

Behavior & response style:
- Prioritize concise, evidence-based recommendations derived from tool outputs.
- If results are insufficient or ambiguous, ask a focused clarifying question (do not call a tool) before searching further.
- Always include practical next steps (e.g., outreach template, suggested interview questions, priority ranking).
- Do not include any explanatory or narrative text when issuing a tool call — only emit the required JSON.

Follow these rules strictly to ensure consistent, parseable tool usage and clear recruiter-oriented recommendations."""

Alongside our system prompt, we must also pass Ollama the schema of our functions for tool calls. [Tool calling](https://ollama.com/blog/tool-support) is a relatively new feature in Ollama and is used by providing function schemas to the `tools` parameter when calling our LLM.

We use `FunctionSchema` object with `to_ollama` from `semantic-router` to transform our functions into correctly formatted schemas.

In [8]:
pip install -qU semantic-router

Note: you may need to restart the kernel to use updated packages.


In [9]:
from semantic_router.utils.function_call import FunctionSchema

# create the function calling schema for ollama
search_schema = FunctionSchema(search).to_ollama()
# TODO deafult None value for description and fix required fields in SR
search_schema["function"]["parameters"]["properties"]["query"]["description"] = None
search_schema

ImportError: cannot import name 'convert_python_type_to_json_type' from partially initialized module 'semantic_router.utils.function_call' (most likely due to a circular import) (c:\YoussefENSI_backup\Eukliadia-test\Git Repo\.venv\Lib\site-packages\semantic_router\utils\function_call.py)

In [11]:
import inspect
import typing
from typing import get_origin, get_args

def _map_python_type(py_type):
    """Map simple python/typing types to JSON Schema types used by Ollama."""
    origin = get_origin(py_type)
    args = get_args(py_type)
    # simple builtin types
    if py_type is str:
        return {"type": "string"}
    if py_type is int:
        return {"type": "integer"}
    if py_type is float:
        return {"type": "number"}
    if py_type is bool:
        return {"type": "boolean"}
    # list[...] -> array with items
    if origin in (list, typing.List):
        item_type = args[0] if args else str
        return {"type": "array", "items": _map_python_type(item_type)}
    # dict[...] -> object (loose)
    if origin in (dict, typing.Dict):
        return {"type": "object"}
    # fallback
    return {"type": "string"}

def func_to_ollama_schema(func, name=None, description=None):
    """
    Build a minimal Ollama function schema from a Python function.
    Returns a dict shaped like: {"function": {"name": ..., "description": ..., "parameters": {...}}}
    - Sets parameter descriptions to None by default (matching your TODO).
    - Marks parameters without defaults as required.
    """
    sig = inspect.signature(func)
    params = {}
    required = []
    for pname, param in sig.parameters.items():
        if param.kind in (inspect.Parameter.VAR_POSITIONAL, inspect.Parameter.VAR_KEYWORD):
            # skip *args/**kwargs
            continue
        # use annotation if present, otherwise assume string
        ann = param.annotation if param.annotation is not inspect._empty else str
        prop_schema = _map_python_type(ann)
        # set description to None to match your notebook TODO
        prop_schema["description"] = None
        params[pname] = prop_schema
        if param.default is inspect._empty:
            required.append(pname)

    parameters = {
        "type": "object",
        "properties": params,
    }
    if required:
        parameters["required"] = required

    return {
        "function": {
            "name": name or func.__name__,
            "description": description if description is not None else (func.__doc__ or None),
            "parameters": parameters,
        }
    }

In [23]:
search_schema = func_to_ollama_schema(linkedin_candidate_scrapper, description=None)
search_schema

{'function': {'name': 'linkedin_candidate_scrapper',
  'description': "\n    Provides access to a LinkedIn candidate scrapping tool.\n    This function simulates a candidate search by returning a list\n    of random candidate recommendations. In a real implementation, it\n    would connect to LinkedIn's API or use web scraping techniques\n    to retrieve candidate profiles.\n    ",
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'string', 'description': None}},
   'required': ['query']}}}

In [25]:
final_answer_schema = func_to_ollama_schema(final_answer, description=None)
final_answer_schema

{'function': {'name': 'final_answer',
  'description': "Returns a natural language response to the user. There are four sections \n    to be returned to the user, those are:\n    - `answer`: the final natural language answer to the user's question, should provide as much context as possible.\n    - `phone_number`: the phone number of top recommended restaurant (if found).\n    - `address`: the address of the top recommended restaurant (if found).\n    ",
  'parameters': {'type': 'object',
   'properties': {'answer': {'type': 'string', 'description': None},
    'phone_number': {'type': 'string', 'description': None},
    'address': {'type': 'string', 'description': None}},
   'required': ['answer']}}}

Now we can test our LLM!

---

**❗️ Make sure you have Ollama running locally and you have already downloaded the model with `ollama pull llama3.1:8b`!**

```

In [26]:
import ollama

def get_system_tools_prompt(system_prompt: str, tools: list[dict]):
    tools_str = "\n".join([str(tool) for tool in tools])
    return (
        f"{system_prompt}\n\n"
        f"You may use the following tools:\n{tools_str}"
    )

res = ollama.chat(
    model="llama3-groq-tool-use:8b",
    messages=[
        {"role": "system", "content": get_system_tools_prompt(
            system_prompt=system_prompt,
            tools=[search_schema, final_answer_schema]
        )},
        # chat history will go here
        {"role": "user", "content": "hello there"}
        # scratchpad will go here
    ],
    format="json",
)

2025-09-29 02:23:22 - httpx - INFO - _client.py:1025 - _send_single_request() - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


In [27]:
res

ChatResponse(model='llama3-groq-tool-use:8b', created_at='2025-09-29T01:23:22.7399572Z', done=True, done_reason='stop', total_duration=83555763200, load_duration=10593596500, prompt_eval_count=502, prompt_eval_duration=68456659000, eval_count=14, eval_duration=4497960500, message=Message(role='assistant', content='{ "name": "final_answer", "parameters": {} }', thinking=None, images=None, tool_name=None, tool_calls=None))

We can see here that the LLM is correctly deciding to use the `final_answer` tool to respond to the user. To parse this information we can use `json.loads`:

In [28]:
import json

json.loads(res["message"]["content"])

{'name': 'final_answer', 'parameters': {}}

Let's see if we can get it to use the reddit search tool.

In [31]:
res = ollama.chat(
    model="llama3-groq-tool-use:8b",
    messages=[
        {"role": "system", "content": get_system_tools_prompt(
            system_prompt=system_prompt,
            tools=[search_schema, final_answer_schema]
        )},
        # chat history will go here
        {"role": "user", "content": "hi, I'm looking for a software engnieer with experience in python and machine learning."}
        # scratchpad will go here
    ],
    format="json",
)
# parse the output
json.loads(res["message"]["content"])

2025-09-29 02:25:54 - httpx - INFO - _client.py:1025 - _send_single_request() - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


{'name': 'linkedin_candidate_scrapper',
 'parameters': {'query': 'software engineer, python, machine learning'}}

Again, this looks perfect!

To keep things a little more organized we can use another pydantic schema to organize the output from our LLM.

In [32]:
class AgentAction(BaseModel):
    tool_name: str
    tool_input: dict
    tool_output: str | None = None

    @classmethod
    def from_ollama(cls, ollama_response: dict):
        try:
            # parse the output
            output = json.loads(ollama_response["message"]["content"])
            return cls(
                tool_name=output["name"],
                tool_input=output["parameters"],
            )
        except Exception as e:
            print(f"Error parsing ollama response:\n{ollama_response}\n")
            raise e

    def __str__(self):
        text = f"Tool: {self.tool_name}\nInput: {self.tool_input}"
        if self.tool_output is not None:
            text += f"\nOutput: {self.tool_output}"
        return text


action = AgentAction.from_ollama(res)
action

AgentAction(tool_name='linkedin_candidate_scrapper', tool_input={'query': 'software engineer, python, machine learning'}, tool_output=None)

That looks great! Now we just need to wrap this with the ability to contain chat history and the agent scratchpad — before adding everything into our graph.

For our agent actions, we will be converting them into fake back-and-forth messages between the assistant and user. For example:

```python
AgentAction(
    tool_name="xyz",
    tool_input={"query": "something cool"},
    tool_output="A fascinating tidbit of information"
)
```

Would become:

```json
[
    {"role": "assistant", "content": "{'name': 'xyz', 'parameters': {'query': 'something cool'}"},
    {"role": "user", "content": "A fascinating tidbit of information"}
]
```

We will make this happen with an `action_to_message` function:

In [33]:
def action_to_message(action: AgentAction):
    # create assistant "input" message
    assistant_content = json.dumps({"name": action.tool_name, "parameters": action.tool_input})
    assistant_message = {"role": "assistant", "content": assistant_content}
    # create user "response" message
    user_message = {"role": "user", "content": action.tool_output}
    return [assistant_message, user_message]

Let's test:

In [34]:
test_action = AgentAction(
    tool_name="xyz",
    tool_input={"query": "something cool"},
    tool_output="A fascinating tidbit of information"
)
action_to_message(test_action)

[{'role': 'assistant',
  'content': '{"name": "xyz", "parameters": {"query": "something cool"}}'},
 {'role': 'user', 'content': 'A fascinating tidbit of information'}]

In [35]:
def create_scratchpad(intermediate_steps: list[AgentAction]):
    # filter for actions that have a tool_output
    intermediate_steps = [action for action in intermediate_steps if action.tool_output is not None]
    # format the intermediate steps into a "assistant" input and "user" response list
    scratch_pad_messages = []
    for action in intermediate_steps:
        scratch_pad_messages.extend(action_to_message(action))
    return scratch_pad_messages

def call_llm(user_input: str, chat_history: list[dict], intermediate_steps: list[AgentAction]) -> AgentAction:
    # format the intermediate steps into a scratchpad
    scratchpad = create_scratchpad(intermediate_steps)
    # if the scratchpad is not empty, we add a small reminder message to the agent
    if scratchpad:
        scratchpad += [{
            "role": "user",
            "content": (
                f"Please continue, as a reminder my query was '{user_input}'. "
                "Only answer to the original query, and nothing else — but use the "
                "information I provided to you to do so. Provide as much "
                "information as possible in the `answer` field of the "
                "final_answer tool and remember to leave the contact details "
                "of a promising looking candidate."
            )
        }]
        # we determine the list of tools available to the agent based on whether
        # or not we have already used the search tool
        tools_used = [action.tool_name for action in intermediate_steps]
        tools = []
        if "search" in tools_used:
            # we do this because the LLM has a tendency to go off the rails
            # and keep searching for the same thing
            tools = [final_answer_schema]
            scratchpad[-1]["content"] = " You must now use the final_answer tool."
        else:
            # this shouldn't happen, but we include it just in case
            tools = [search_schema, final_answer_schema]
    else:
        # this would indiciate we are on the first run, in which case we
        # allow all tools to be used
        tools = [search_schema, final_answer_schema]
    # construct our list of messages
    messages = [
        {"role": "system", "content": get_system_tools_prompt(
            system_prompt=system_prompt,
            tools=tools
        )},
        *chat_history,
        {"role": "user", "content": user_input},
        *scratchpad,
    ]
    res = ollama.chat(
        model="llama3-groq-tool-use:8b",
        messages=messages,
        format="json",
    )
    return AgentAction.from_ollama(res)

Let's try `call_llm` *with* chat history:

In [36]:
# let's fake some chat history and test
out = call_llm(
    chat_history=[
        {"role": "user", "content": "hi im looking for a software engineer"},
        {"role": "assistant", "content": "okey great, can you tell me more about the role?"},
        {"role": "user", "content": "the role is for a backend python developer with experience in machine learning"},
        {"role": "assistant", "content": "Okay, I will start looking for candidates."},
    ],
    user_input="okey perfect, please go ahead",
    intermediate_steps=[]
)
out

2025-09-29 02:29:18 - httpx - INFO - _client.py:1025 - _send_single_request() - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


AgentAction(tool_name='linkedin_candidate_scrapper', tool_input={'query': 'backend Python developer with machine learning experience'}, tool_output=None)

We intentionally didn't include where we are in our current `user_input`, but instead included it in the `chat_history` to confirm our agent is able to consider the chat history when building our web search tool call. It succeeded! Now we can move on to constructing our graph.

Once we have our search query, we can pass it onto our `search` tool to get some results, let's try:

In [37]:
results = linkedin_candidate_scrapper(**out.tool_input)
print(results)

Searching for candidates with query: backend Python developer with machine learning experience
[Rec(title='Ella Davis', description='Sales professional with a proven track record in exceeding targets.', comments=['Candidate Ella Davis demonstrates strong potential in their field.']), Rec(title='Ella Davis', description='Sales professional with a proven track record in exceeding targets.', comments=['Candidate Ella Davis demonstrates strong potential in their field.']), Rec(title='Alice Johnson', description='Experienced software engineer with expertise in Python and Machine Learning.', comments=['Candidate Alice Johnson demonstrates strong potential in their field.'])]


We now have our results! Each of these is pretty high level, there is not much detail as they only represent search page result descriptions. So now, we must decide which links look most promising — we can do that by passing these results onwards to another LLM that decides which result we should get more information from.

### Graph Nodes

We have defined the different logical components of our graph, but we need to execute them in a langgraph-friendly manner — for that they must consume our `AgentState` and return modifications to that state. We will do this for all of our components via three functions:

* `run_oracle` will handle running our oracle LLM.
* `router` will handle the *routing* between our oracle and tools.
* `run_tool` will handle running our tool functions.

In [38]:
def run_oracle(state: TypedDict):
    print("run_oracle")
    chat_history = state["chat_history"]
    out = call_llm(
        user_input=state["input"],
        chat_history=chat_history,
        intermediate_steps=state["intermediate_steps"]
    )
    return {
        "intermediate_steps": [out]
    }

def router(state: TypedDict):
    print("router")
    # return the tool name to use
    if isinstance(state["intermediate_steps"], list):
        return state["intermediate_steps"][-1].tool_name
    else:
        # if we output bad format go to final answer
        print("Router invalid format")
        return "final_answer"

# we use this to map tool names to tool functions
tool_str_to_func = {
    "linkedin_candidate_scrapper": linkedin_candidate_scrapper,
    "final_answer": final_answer
}

def run_tool(state: TypedDict):
    # use this as helper function so we repeat less code
    tool_name = state["intermediate_steps"][-1].tool_name
    tool_args = state["intermediate_steps"][-1].tool_input
    print(f"run_tool | {tool_name}.invoke(input={tool_args})")
    # run tool
    out = tool_str_to_func[tool_name](**tool_args)
    action_out = AgentAction(
        tool_name=tool_name,
        tool_input=tool_args,
        tool_output=str(out),
    )
    if tool_name == "final_answer":
        return {"output": out}
    else:
        return {"intermediate_steps": [action_out]}

We construct our graph using `add_nodes`, `add_edge`, and `add_conditional_edges`.

In [40]:
from langgraph.graph import StateGraph, END

graph = StateGraph(AgentState)

graph.add_node("oracle", run_oracle)
graph.add_node("linkedin_candidate_scrapper", run_tool)
graph.add_node("final_answer", run_tool)

graph.set_entry_point("oracle")  # insert query here

graph.add_conditional_edges(  # - - - >
    source="oracle",  # where in graph to start
    path=router,  # function to determine which node is called
)

# create edges from each tool back to the oracle
for tool_obj in [search_schema, final_answer_schema]:
    tool_name = tool_obj["function"]["name"]
    if tool_name != "final_answer":
        graph.add_edge(tool_name, "oracle")  # ————————>

# if anything goes to final answer, it must then move to END
graph.add_edge("final_answer", END)

runnable = graph.compile()

To view the graph we can generate a mermaid graph like so:

In [41]:
import nest_asyncio
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

nest_asyncio.apply()  # Required for Jupyter Notebook to run async functions

display(
    Image(
        runnable.get_graph().draw_mermaid_png(
            curve_style=CurveStyle.LINEAR,
            node_colors=NodeStyles(first="#ffdfba", last="#baffc9", default="#fad7de"),
            wrap_label_n_words=9,
            output_file_path=None,
            draw_method=MermaidDrawMethod.PYPPETEER,
            background_color="white",
            padding=10,
        )
    )
)

[INFO] Starting Chromium download.
2025-09-29 02:30:59 - pyppeteer.chromium_downloader - INFO - chromium_downloader.py:75 - download_zip() - Starting Chromium download.


OSError: Chromium downloadable not found at https://storage.googleapis.com/chromium-browser-snapshots/Win_x64/1181205/chrome-win.zip: Received <?xml version='1.0' encoding='UTF-8'?><Error><Code>NoSuchKey</Code><Message>The specified key does not exist.</Message><Details>No such object: chromium-browser-snapshots/Win_x64/1181205/chrome-win.zip</Details></Error>.


## Testing the Agent

Our agent has now been constructed so we can test it. First, let's check for the best pizza in Rome:

In [42]:
out = runnable.invoke({
    "input": "hi im looking for a software engineer",
    "chat_history": [],
})

run_oracle


2025-09-29 02:31:21 - httpx - INFO - _client.py:1025 - _send_single_request() - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


router
run_tool | linkedin_candidate_scrapper.invoke(input={'query': 'software engineer'})
Searching for candidates with query: software engineer
run_oracle


2025-09-29 02:32:08 - httpx - INFO - _client.py:1025 - _send_single_request() - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


router
run_tool | final_answer.invoke(input={'answer': 'Based on your search, I found three strong candidates: Carol Williams, Ella Davis, and David Brown. Each has demonstrated significant potential in their field. It might be worth exploring how they can contribute to your project.'})


We extract the answer:

In [28]:
out["output"]

{'answer': 'Based on your question, I would recommend trying Seu Pizza Illuminati in Rome for an amazing pizza experience. They are known for their creative use of condiments and experiments with vegetables. If you prefer a Neapolitan-style pizza, Pizzeria da Remo or Emma might be the perfect choice for you. Both places have received great reviews from locals and visitors alike.',
 'phone_number': '',
 'address': ''}

We are recommended [Seu Pizza Illuminati](https://maps.app.goo.gl/RMSdTUpH8D3oQETUA), a seemingly notorious pizzeria known for their less traditional and more experimental Neapolitan pizzas.

---